In [83]:
import psycopg2
conn = psycopg2.connect('postgresql://brosra01:@localhost/world')
cur = conn.cursor()
cur.execute('select * from country limit 10')
res = cur.fetchall()
#for row in res[:5]:
    #print(row)
    

In [84]:
import csv
import psycopg2
conn = psycopg2.connect('postgresql://brosra01:@localhost/world')
cur = conn.cursor()
x = 'USA'
cur.execute("select code, name, region, continent from country where code = '%s'" % x)
res = cur.fetchall()
#for row in res:
 #   print(row)

    
        

In [85]:
import csv
import psycopg2

with open('/home/faculty/millbr02/pub/un-general-debates.csv','r',encoding='latin-1') as data:
    csv_iter = csv.reader(data)
    db =[]
    for row in csv_iter:
        db.append(row)

In [86]:
with open('/home/faculty/millbr02/pub/un-debates-augmented-1.csv','r',encoding='latin-1') as data:
    csv_iter = csv.reader(data)
    db = []
    for row in csv_iter:
        db.append(row)
    
        

    country_codes = {}
    for line in db[1:]:
        country_codes[line[4]] = line[2]
        


        
    country_names = {}
    for line in db[1:]:
        country_names[line[4]] = 0
        
        



    newlist = []
    for line in db[1:]:
        newlist.append(line[4])


    
    matrix = {}
    for row in db[1:]:
        spkr = row[4]
        text = row[3]
        if spkr not in matrix:
            matrix[spkr] = country_names.copy()
            
            
        
        for key in matrix:
            counter = text.count(key)
            matrix[spkr][key] += counter      
        

    all_mentions = {}
    for key in matrix:
        for key2 in matrix[key]:
        #all_mentions.append(matrix[key])
            if key2 not in all_mentions:
                #if key == key2:
                #    all_mentions[key2] = [int(matrix[key][key2])]
            
                all_mentions[key2] = [int(matrix[key][key2])]
            else:
                all_mentions[key2].append(int(matrix[key][key2]))
                

    all_count = {}        
    for key in all_mentions:
        count = sum(all_mentions[key])
        all_count[key] = count
        
        
        
#Uncomment to display information accordingly:
        
all_count
#country_codes
#country_names
#newlist
#matrix
#all_mentions
    


{'Afghanistan': 6052,
 'Albania': 1443,
 'Algeria': 964,
 'Andorra': 362,
 'Angola': 4158,
 'Antigua and Barbuda': 322,
 'Argentina': 1204,
 'Armenia': 870,
 'Australia': 1572,
 'Austria': 991,
 'Azerbaijan': 1094,
 'Bahamas': 1083,
 'Bahrain': 608,
 'Bangladesh': 1282,
 'Barbados': 1185,
 'Belarus': 519,
 'Belgium': 556,
 'Belize': 1090,
 'Benin': 630,
 'Bhutan': 389,
 'Bolivia': 1370,
 'Bosnia and Herzegovina': 1122,
 'Botswana': 482,
 'Brazil': 1552,
 'Brunei': 280,
 'Bulgaria': 1040,
 'Burkina Faso': 375,
 'Burundi': 1702,
 'Cambodia': 2799,
 'Cameroon': 498,
 'Canada': 1133,
 'Cape Verde': 514,
 'Central African Republic': 646,
 'Chad': 2118,
 'Chile': 1862,
 'China': 4780,
 'Colombia': 1146,
 'Comoros': 662,
 'Congo': 2142,
 'Congo, The Democratic Republic of the': 0,
 'Costa Rica': 858,
 'Croatia': 954,
 'Cuba': 2816,
 'Cyprus': 4339,
 'Czech Republic': 295,
 'Czechoslovakia': 246,
 'CÃ´te dÂ\x92Ivoire': 0,
 'Democratic Yemen': 52,
 'Denmark': 511,
 'Djibouti': 540,
 'Dominica':

In [87]:
%matplotlib inline
import networkx as nx
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [88]:
g = nx.Graph()

In [89]:
g.add_nodes_from(matrix) 
for key in matrix: 
    for key2 in matrix[key]: 
        if matrix[key][key2] > 50:
            g.add_edge(key, key2, weight=matrix[key][key2]) 


In [90]:
#gfig, gax = plt.subplots(figsize = (50,50))
hego = nx.ego_graph(g,'Egypt', radius =2)
pos = nx.spring_layout(g, k=.3, iterations=2000, scale =10)
#nx.draw(hego, pos, ax=gax, node_size=200, with_labels=True, font_size=40)

In [91]:
import requests

In [92]:
result = requests.get("http://localhost/~brosra01/index.html")

In [93]:
result.status_code

200

In [94]:
result

<Response [200]>

In [95]:
result = requests.get("https://opendata.socrata.com/api/views/mnkm-8ram/rows.json?accessType=DOWNLOAD")

In [96]:
j = result.json()

In [97]:
j.keys()

dict_keys(['data', 'meta'])

In [98]:
latlong = {}
for item in j['data']:
    latlong[item[10]] = {'Latitude': item[12], 'Longitude': item[13]} 

#latlong



In [99]:
import csv
import psycopg2

newlist = []

with open('/home/faculty/millbr02/pub/un-debates-augmented-1.csv','r',encoding='latin-1') as data:
    csv_iter = csv.reader(data)
    db =[]
    for row in csv_iter:
        db.append(row)
    
    for line in db:
        for key in latlong:
            if line[2] == key:
                line.insert(3, (latlong[key]['Latitude']))
                line.insert(4, (latlong[key]['Longitude']))
        for k in all_count:
            if line[6] == k:
                line.insert(6, (str(all_count[k])))
    
    
    for line in db:
        if len(line) < 9:
            del line[3]
        del line[0]
        del line[0]
        del line[3]
        line[1] = ''.join(line[1])
        line[2] = ''.join(line[2])
        if len(line) > 3:
            line[3] = ''.join(line[3])
        newlist.append(line)
    
    checking_for_duplicates = []
    final_list = []
    for line in newlist:
        if line[0] not in checking_for_duplicates:
            checking_for_duplicates.append(line[0])
            final_list.append(line)
        else:
            pass
            
        

        
'''Each line in this list contains: 3-Letter-CountryCode, Latitude, Longitude, Count of mentions,
Country name, Region, Continent.'''



final_list[0] = ['Countrycode','Latitude','Longitude','Totalmentions','Countryname','Region','Continent']
final_list

[['Countrycode',
  'Latitude',
  'Longitude',
  'Totalmentions',
  'Countryname',
  'Region',
  'Continent'],
 ['MDV', '3.25', '73', '485', 'Maldives', 'Southern and Central Asia', 'Asia'],
 ['FIN', '64', '26', '551', 'Finland', 'Nordic Countries', 'Europe'],
 ['NER', '16', '8', '1761', 'Niger', 'Western Africa', 'Africa'],
 ['URY', '-33', '-56', '1262', 'Uruguay', 'South America', 'South America'],
 ['ZWE', '-20', '30', '2581', 'Zimbabwe', 'Eastern Africa', 'Africa'],
 ['PHL', '13', '122', '541', 'Philippines', 'Southeast Asia', 'Asia'],
 ['SDN', '15', '30', '2631', 'Sudan', 'Northern Africa', 'Africa'],
 ['RUS', '60', '100', '325', 'Russian Federation', 'Eastern Europe', 'Europe'],
 ['CHN', '35', '105', '4780', 'China', 'Eastern Asia', 'Asia'],
 ['ESP', '40', '-4', '1421', 'Spain', 'Southern Europe', 'Europe'],
 ['SUR', '4', '-56', '507', 'Suriname', 'South America', 'South America'],
 ['ARG', '-34', '-64', '1204', 'Argentina', 'South America', 'South America'],
 ['SLV',
  '13.8333',

In [100]:
with open('augmentedwithlatituteandlongitude.csv', 'w', encoding='latin-1') as data:
    for line in final_list:
        data.write(', '.join(line)+'\n')

In [101]:
with open('augmentedwithlatituteandlongitude.csv', 'r', encoding='latin-1') as data:
    for line in data:
        print(line)

Countrycode, Latitude, Longitude, Totalmentions, Countryname, Region, Continent

MDV, 3.25, 73, 485, Maldives, Southern and Central Asia, Asia

FIN, 64, 26, 551, Finland, Nordic Countries, Europe

NER, 16, 8, 1761, Niger, Western Africa, Africa

URY, -33, -56, 1262, Uruguay, South America, South America

ZWE, -20, 30, 2581, Zimbabwe, Eastern Africa, Africa

PHL, 13, 122, 541, Philippines, Southeast Asia, Asia

SDN, 15, 30, 2631, Sudan, Northern Africa, Africa

RUS, 60, 100, 325, Russian Federation, Eastern Europe, Europe

CHN, 35, 105, 4780, China, Eastern Asia, Asia

ESP, 40, -4, 1421, Spain, Southern Europe, Europe

SUR, 4, -56, 507, Suriname, South America, South America

ARG, -34, -64, 1204, Argentina, South America, South America

SLV, 13.8333, -88.9167, 1097, El Salvador, Central America, North America

MYS, 2.5, 112.5, 783, Malaysia, Southeast Asia, Asia

NPL, 28, 84, 894, Nepal, Southern and Central Asia, Asia

PRT, 39.5, -8, 1770, Portugal, Southern Europe, Europe

COL, 4, -72

In [102]:
result.iter_lines()

<generator object Response.iter_lines at 0x7f191e45d990>

In [103]:
#for line in result.iter_lines():
 #   print(line.decode('utf-8'))